# Creating and Valuing a CDS Contract

Creates a CDS contract and does a valuation and calculates risk measures

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating a CDS Contract

In [3]:
valuationDate = TuringDate(9, 8, 2019)
effectiveDate = valuationDate
maturityDate = TuringDate(20, 6, 2029)
cdsCoupon = 0.0150
notional = ONE_MILLION
longProtection = False
tradeDate = TuringDate(9, 8, 2019)

In [4]:
cdsContract = FinCDS(effectiveDate, maturityDate, cdsCoupon, notional, longProtection)

## Build Ibor Curve

In [5]:
settlementDate = effectiveDate

In [12]:
dcType = TuringDayCountTypes.ACT_360
depo1 = TuringIborDeposit(settlementDate, "1M", 0.022009, dcType)
depo2 = TuringIborDeposit(settlementDate, "2M", 0.022138, dcType)
depo3 = TuringIborDeposit(settlementDate, "3M", 0.021810, dcType)
depo4 = TuringIborDeposit(settlementDate, "6M", 0.020503, dcType)
depo5 = TuringIborDeposit(settlementDate, "12M", 0.019930, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = TuringDayCountTypes.THIRTY_E_360_ISDA
fixedFreq = TuringFrequencyTypes.SEMI_ANNUAL
swapType = TuringSwapTypes.PAY

swap1 = FinIborSwap(settlementDate,"2Y",swapType,0.015910,fixedFreq,dcType)
swap2 = FinIborSwap(settlementDate,"3Y",swapType,0.014990,fixedFreq,dcType)
swap3 = FinIborSwap(settlementDate,"4Y",swapType,0.014725,fixedFreq,dcType)
swap4 = FinIborSwap(settlementDate,"5Y",swapType,0.014640,fixedFreq,dcType)
swap5 = FinIborSwap(settlementDate,"6Y",swapType,0.014800,fixedFreq,dcType)
swap6 = FinIborSwap(settlementDate,"7Y",swapType,0.014995,fixedFreq,dcType)
swap7 = FinIborSwap(settlementDate,"8Y",swapType,0.015180,fixedFreq,dcType)
swap8 = FinIborSwap(settlementDate,"9Y",swapType,0.015610,fixedFreq,dcType)
swap9 = FinIborSwap(settlementDate,"10Y",swapType,0.0159880,fixedFreq,dcType)
swap10 = FinIborSwap(settlementDate,"12Y",swapType,0.016430,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

liborCurve = TuringIborSingleCurve(valuationDate, depos, [], swaps)

# Build a CDS Curve

In [13]:
cds1 = FinCDS(settlementDate, "1Y", 0.0200)
cds2 = FinCDS(settlementDate, "2Y", 0.0220)
cds3 = FinCDS(settlementDate, "3Y", 0.0250)
cds4 = FinCDS(settlementDate, "4Y", 0.0275)
cds5 = FinCDS(settlementDate, "5Y", 0.0290)
cds6 = FinCDS(settlementDate, "7Y", 0.0300)
cds7 = FinCDS(settlementDate, "10Y", 0.0310)
cds8 = FinCDS(settlementDate, "15Y", 0.0315)

In [14]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [15]:
recoveryRate = 0.40

In [16]:
issuerCurve = FinCDSCurve(valuationDate, cdss, liborCurve, recoveryRate)

In [17]:
print(issuerCurve)

OBJECT TYPE: FinCDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 1.1178082,  0.9629280
 2.1178082,  0.9241524
 3.1178082,  0.8756501
 4.1178082,  0.8234894
 5.1205479,  0.7743682
 7.1205479,  0.6920884
10.1232877,  0.5813178
15.1260274,  0.4389424


# Valuation

In [18]:
spd = cdsContract.parSpread(settlementDate, issuerCurve, recoveryRate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  309.40693 bp


In [19]:
v = cdsContract.value(settlementDate, issuerCurve, recoveryRate)

In [20]:
full_pv = v['full_pv'] 
clean_pv = v['clean_pv']

In [21]:
print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -115810.02
CLEAN VALUE   -117893.35


In [22]:
cleanp = cdsContract.cleanPrice(settlementDate, issuerCurve, recoveryRate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE    88.210959


In [23]:
accruedDays = cdsContract.accruedDays()
print("ACCRUED_DAYS", accruedDays)

ACCRUED_DAYS 50


In [24]:
accruedInterest = cdsContract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON 2083.3333333333335


In [20]:
protPV = cdsContract.protectionLegPV(settlementDate, issuerCurve, recoveryRate)
print("PROTECTION_PV", protPV)

PROTECTION_PV 228829.57245128392


In [21]:
premPV = cdsContract.premiumLegPV(settlementDate, issuerCurve, recoveryRate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 113019.55526164455


In [22]:
rpv01 = cdsContract.riskyPV01(settlementDate, issuerCurve)
print("FULL_RPV01", rpv01['full_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

FULL_RPV01 7.534637017442971
CLEAN_RPV01 7.395748128554081


In [23]:
cdsContract.riskyPV01(settlementDate, issuerCurve)

{'full_rpv01': 7.534637017442971, 'clean_rpv01': 7.395748128554081}

## Risk Measures

In [24]:
creditDV01 = cdsContract.creditDV01(settlementDate, issuerCurve, recoveryRate)

In [25]:
creditDV01

-650.8402049520955

Copyright (c) 2020 Dominic O'Kane